In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
from library import Plugin, CacheFiles
from library import get_help
from importlib import import_module
from pathlib import Path
import constants
import logging
from IPython.display import Image 


In [17]:
logging.root.setLevel('WARNING')

In [58]:
# discover plugins
plugins = get_help.get_modules()

# set up basic configuration
cache = CacheFiles()
active_plugins = []
refresh_rate = 1
plugin_path = Path('./plugins')
resolution = (640,448)
plugin_readme_name = f'README.md'

# loop through all the discovered plugins
for plugin in plugins:
    config = None
    module = None
    doc_path = Path(plugin_path/f'{plugin}/')
    plugin_sample_img = None
    try:
        sample = import_module(f'{constants.plugins}.{plugin}.sample')
        module = import_module(f'{constants.plugins}.{plugin}')
    except ModuleNotFoundError:
        logging.warning(f'module "{plugin}" is missing a sample configuration; cannot process')
        continue
        
    config = sample.config        
    if config and module:
        layout = getattr(module.layout, config['layout'])
        my_plugin = Plugin(resolution=resolution, 
                           cache=cache, layout=layout,
                           update_function=module.update_function, config=config)
        my_plugin.refresh_rate = refresh_rate
        active_plugins.append(my_plugin)
        if 'kwargs' in config:
            my_plugin.update(**config['kwargs'])
        else:
            my_plugin.update()
#         display(my_plugin.image)
    
    # write out readme and save image
    plugin_readme = Path(doc_path/plugin_readme_name)
    if my_plugin.image:
        plugin_sample_img = Path(doc_path/f'{plugin}_sample.png')
        my_plugin.image.save(plugin_sample_img)
    with open(plugin_readme, 'w') as file:
        file.write(f'# {plugin}\n')
        file.write(f'![sample image for plugin {plugin}]({plugin_sample_img.name})\n')
        file.write('```\n' + get_help.get_help(plugin, False) + '\n```')
#         print(f"## {mod_name} ")
#         print('```\n' + get_help.get_module_docs(module) + '\n```')

    